In [1]:
import time
import os
from configparser import ConfigParser
import paramiko

from camtune.database import PostgresqlDB
from camtune.search_space import SearchSpace
from camtune.utils.logger import load_logger
from camtune.utils.parser import parse_pgbench_output

DEFAULT_CONFIG_FILE = '/home/viktor/Experiments/CamDB/camtune/config/postgre_tpch_remote.yaml'
BENCHMARK_DIR = '/home/viktor/Experiments/CamDB/camtune/benchmarks'

In [2]:
import yaml
with open(DEFAULT_CONFIG_FILE, 'r') as f:
    config = yaml.safe_load(f)

logger = load_logger(config)
db = PostgresqlDB(config, param_logger=logger)

res = {'benchmark': db.benchmark}
total_exec_time = 0
query_file_names = []

if not db.benchmark_fast:
    query_list_file = 'tpch_query_list.txt'
else:
    query_list_file = 'tpch_query_fast_list.txt'
print(f"[PostgresqlDB] Exeucting queries listed in {query_list_file}")

lines = open(os.path.join(BENCHMARK_DIR, query_list_file), 'r').readlines()
for line in lines:
    query_file_names.append(line.rstrip())
query_file_names = [os.path.join(db.query_dir, query_file_name) for query_file_name in query_file_names]

query_file_names

[PostgresqlDB] Exeucting queries listed in tpch_query_fast_list.txt


['/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/1.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/2.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/3.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/4.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/5.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/6.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/7.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/8.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/9.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/10.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/11.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/12.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/13.sql',
 '/home/viktor/Experiments/CamDB/camtune/benchm

In [3]:
search_space = SearchSpace(
    config['tune']['knob_definitions'], 
    is_kv_config=True,
    seed=1,
)

sample_configuration = search_space.input_space.sample_configuration()
# sample_configuration
db.apply_knobs_offline(sample_configuration)

res = db.exec_queries(['/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/18.sql'])


[PostgresqlDB] Error information: SSL SYSCALL error: EOF detected



In [4]:
res

({}, ['/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/18.sql'])

In [31]:
db_name = 'tpch'
res: dict = {}
failed = []

remote_tmp_sql = '/tmp/tmp.sql'

# Build SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(db.host, username=db.ssh_user, password=db.ssh_pwd,
            disabled_algorithms={'pubkeys': ['rsa-sha2-256', 'rsa-sha2-512']})

start, end = 0, 17
for i, query_file_name in enumerate(query_file_names[start:end]):
    print(query_file_name)
    sftp = ssh.open_sftp()
    try:
        # Put local query file into a temporary file in remote server
        sftp.put(query_file_name, remote_tmp_sql)
    except IOError:
        logger.info(f'[PostgresqlDB] Remote SFTP put SQL query {query_file_name} failed')
        if sftp: sftp.close()
        continue
    
    if sftp: sftp.close()
    # Execute pgbench command
    command = f"pgbench -f {remote_tmp_sql} {db_name} -n"
    _, stdout, _ = ssh.exec_command(command)
    
    retcode = stdout.channel.recv_exit_status()
    if retcode != 0:
        logger.info(f'[PostgresqlDB] Remote executing SQL query {query_file_name} using pgbench failed')
        failed.append(query_file_name)
        continue

    output = stdout.read().decode('utf-8')
    res[query_file_name] = parse_pgbench_output(output)
    logger.info(f'[PostgresqlDB] Remote executing SQL query {query_file_name} using pgbench successfully')
                            
if ssh: ssh.close()

/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/1.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/2.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/3.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/4.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/5.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/6.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/7.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/8.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/9.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/10.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/11.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/12.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/13.sql
/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/14.sql
/home/viktor/Experiments/CamD

In [29]:
for i, query_file_name in enumerate(query_file_names[start:end]):
    if i >= 1: break

    print(res[query_file_name])

{'transaction_type': '/tmp/tmp.sql', 'scaling_factor': '1', 'query_mode': 'simple', 'number_of_clients': '1', 'number_of_threads': '1', 'number_of_transactions': '10', 'failed_transactions': '0', 'latency_average': '166.332', 'initial_connection_time': '2.226', 'tps': '6.012069'}


In [32]:
res

{'/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/1.sql': {'transaction_type': '/tmp/tmp.sql',
  'scaling_factor': '1',
  'query_mode': 'simple',
  'number_of_clients': '1',
  'number_of_threads': '1',
  'number_of_transactions': '10',
  'failed_transactions': '0',
  'latency_average': '1260.574',
  'initial_connection_time': '1.979',
  'tps': '0.793290'},
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/2.sql': {'transaction_type': '/tmp/tmp.sql',
  'scaling_factor': '1',
  'query_mode': 'simple',
  'number_of_clients': '1',
  'number_of_threads': '1',
  'number_of_transactions': '10',
  'failed_transactions': '0',
  'latency_average': '145.708',
  'initial_connection_time': '3.319',
  'tps': '6.863046'},
 '/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch_pgbench/3.sql': {'transaction_type': '/tmp/tmp.sql',
  'scaling_factor': '1',
  'query_mode': 'simple',
  'number_of_clients': '1',
  'number_of_threads': '1',
  'number_of_transactions': '10',
  

In [11]:
float(res[query_file_name]['latency_average'])

170.316

In [18]:
# Test a single SFTP query
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(db.host, username=db.ssh_user, password=db.ssh_pwd,
            disabled_algorithms={'pubkeys': ['rsa-sha2-256', 'rsa-sha2-512']})

sftp = ssh.open_sftp()
sftp.put(query_file_name, remote_tmp_sql)
if sftp: sftp.close()


if ssh: ssh.close()

In [19]:
query_file_name

'/home/viktor/Experiments/CamDB/camtune/benchmarks/tpch/19.sql'